<a href="https://colab.research.google.com/github/EdisonVazquezG/DeepLearning_Bourbaki/blob/main/AI_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objetivo

Quieres que un modelo chico (**student**) aprenda a predecir el **siguiente token** como lo haría un modelo guía (**teacher**).

En un batch tienes una secuencia de tokens:

$$
x_1, x_2, \ldots, x_n
$$

y el objetivo de un **LM causal** es:

$$
p\left(x_{t+1}\mid x_{\le t}\right)
$$

El **teacher** ya produce una distribución “buena” sobre el siguiente token.  
El **student** quiere imitarla.


bitsandbytes (bnb) es una librería que permite cargar y usar modelos grandes en GPU con cuantización y optimizaciones para ahorrar VRAM (memoria) y, a veces, acelerar.

¿Qué significa “cuantizar” a 8-bit o 4-bit?

Normalmente los pesos del modelo están en FP16/BF16 (16 bits) o FP32 (32 bits).

  * Cuantizar significa guardarlos/operarlos con menos bits:

  * 8-bit (int8): ~la mitad de memoria vs FP16

  * 4-bit (NF4/FP4): aún menos, típicamente ~1/4 de FP16 (o mejor)

Resultado: un modelo que no cabía en tu GPU ahora sí cabe.

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 15.3 MB/s eta 0:00:00


In [3]:

import os, math, random, warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset, Dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
                          Trainer, TrainingArguments)

1. AutoModelForCausalLM

Qué es: un “loader” automático para modelos de lenguaje autoregresivos (causal LM), tipo GPT-2, LLaMA, Mistral, etc. ¿Por qué “CausalLM”?
Porque el objetivo es predecir el siguiente token usando solo el pasado (máscara causal).

2. BitsAndBytesConfig

Qué es: una configuración para decirle a Transformers cómo cargar el modelo con cuantización usando bitsandbytes (4-bit u 8-bit).

3. Trainer

Qué es: un “motor de entrenamiento” de HuggingFace que te ahorra escribir el loop (forward/backward/optimizer/logging/eval/save).

4. TrainingArguments

Qué es: el objeto donde pones toda la configuración del entrenamiento: batch size, LR, fp16/bf16, cada cuántos pasos evalúas/guardas, etc.

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
cap = torch.cuda.get_device_capability(0)[0] if torch.cuda.is_available() else 0
CAN_BF16 = torch.cuda.is_available() and cap >= 8  # Ampere+ supports bf16
print(f"Device: {device}, CC Major: {cap}, bf16: {CAN_BF16}")

Device: cuda, CC Major: 7, bf16: False


In [5]:
FREE = True  # set False if you have Pro-tier GPU (A100/H100/etc.)

if FREE:
    TEACHER_MODEL = "gpt2"
    STUDENT_MODEL = "distilgpt2" ## Ventaja: comparten vocab/tokenizer, así que comparar logits (KL) es coherente.
    LOAD_IN_4BIT  = False                # 8-bit teacher for T4
    BLOCK_SIZE    = 256 ## menos memoria, más rapido, menos contexto
    TRAIN_TOKENS  = 200_000              # small demo
    VAL_TOKENS    = 20_000
    BATCH_SIZE    = 2 ## cuantas secuencias caben por paso en GPU
    GRAD_ACCUM    = 8 ## cuantos pasos acumulas antes de hacer optimizer.step()
    EPOCHS        = 1
else:
    TEACHER_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"
    STUDENT_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    LOAD_IN_4BIT  = True                 # 4-bit teacher
    BLOCK_SIZE    = 512
    TRAIN_TOKENS  = 500_000
    VAL_TOKENS    = 50_000
    BATCH_SIZE    = 1
    GRAD_ACCUM    = 16 ## Se usa para simular un batch grande cuanto tu GPU no puede cargarlo de golpe. En vez de actualizar los pesos cada batch, haces varios mini-batches, sumas sus gradientes, y solo entonces haces el update.
    EPOCHS        = 1

Qué tan fuerte aprende el student y a quién le hace caso (labels vs teacher).

* BETA = 0.8 (peso de KL / KD)

  KL compara distribuciones teacher vs student (a temperatura T).
  KL dice: “imita al teacher”.

  Con BETA=0.8 estás diciendo:

  “La prioridad es copiar el comportamiento del teacher.”

  Esto suele ser útil cuando:
    * el teacher es fuerte,

    * el student tiene menos capacidad,

    * entrenas poco (demo),

    * quieres transferir “estilo”/regularidades del teacher.

L = (1−λ)·CE + λ·KL

¿Qué pasa si alpha > beta? o ¿al revés? ¿qué es lo que estoy buscando o la motivación?

In [6]:
LR     = 2e-4
T      = 2.0     # KD temperature. Teacher reparte más probabilidad al top-2/top-3… (más información).
ALPHA  = 0.2     # CE weight, ponle atención a los labels reales (ground truth), cross-entropy (perdida de contribución de los labels reales del dataset)
BETA   = 0.8     # KD weight, lambda
OUT_DIR = "kd-out"

In [7]:
if LOAD_IN_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if CAN_BF16 else torch.float16
    )
else:
    bnb_config = BitsAndBytesConfig(load_in_8bit=True)

In [8]:
# Teacher (frozen)
teacher_tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL, use_fast=True) # descargamps el tokenizador y convertimos
teacher_tokenizer.padding_side = "left" # si haces padding lo haces a la izquierda, esto es normal en modelo autoregresivos cuando se quiere alinear los tokens mas recientes al final
if teacher_tokenizer.pad_token is None:
    teacher_tokenizer.pad_token = teacher_tokenizer.eos_token ## con que tokens rellenas el padding, el eos_taken: tomas el token final para rellenar

teacher = AutoModelForCausalLM.from_pretrained( ## cargamos un modelo autoregresivo (aplicamos quantizador)
    TEACHER_MODEL,
    quantization_config=bnb_config,
    device_map="auto" #reparte el modelo entre gpu/cpu
)
teacher.eval()  ## desactivas cosas como: dropout, ciertos comportamientos de entrenamiento.Esto se hace porque quieres que el teacher sea determinista/estable como referencia.
for p in teacher.parameters(): ## congelamos al teacher, no se actualiza, no guardamos gradientes, todo el aprendizaje ocurre en el student
    p.requires_grad_(False)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:

# Student
student_tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL, use_fast=True)
student_tokenizer.padding_side = "left"
if student_tokenizer.pad_token is None:
    student_tokenizer.pad_token = student_tokenizer.eos_token

student = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL,
    torch_dtype=(torch.bfloat16 if CAN_BF16 else (torch.float16 if torch.cuda.is_available() else torch.float32)),
    device_map="auto"
)
#En modelos autoregresivos, el “cache” (past key values) guarda 𝐾,𝑉 de atención de tokens anteriores para acelerar generación.
# con secuencias completas normalmente no necesitas caché.
student.config.use_cache = False  # safer for training

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Congelas al modelo estudiante y solo tomas parte de sus pesos, eso hace LoRA

LoRA normalmente se inserta en capas lineales de un Transformer, sobre todo en:

1) Atención:
   - Proyecciones Q, K, V, O
   - En GPT-2 esto suele estar empaquetado como `c_attn` (QKV juntos) y `c_proj` (proyección de salida)

2) MLP (feed-forward):
   - En LLaMA/Mistral: `up_proj`, `down_proj`, `gate_proj`
   - En GPT-2: `c_fc` (expansión) y `c_proj` (proyección)

Por eso `pick_lora_targets_for_decoder` elige típicamente:
- GPT-2 / DistilGPT-2: `c_attn`, `c_fc`, `c_proj`
- LLaMA / Mistral: `q_proj`, `k_proj`, `v_proj`, `o_proj`, `gate_proj`, `up_proj`, `down_proj`


In [10]:
from peft import LoraConfig, get_peft_model

def pick_lora_targets_for_decoder(model: nn.Module):
    names = [n for n, m in model.named_modules() if isinstance(m, nn.Linear)]
    if any("q_proj" in n for n in names):  # LLaMA/Mistral
        return dict(targets=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
                    fan_in_fan_out=False)
    if any("c_attn" in n for n in names):  # GPT-2 / DistilGPT2
        return dict(targets=["c_attn","c_fc","c_proj"], fan_in_fan_out=True)
    uniq = list({n.split(".")[-1] for n in names})
    return dict(targets=uniq, fan_in_fan_out=False)

In [11]:
cfg = pick_lora_targets_for_decoder(student) #dónde insertar LoRA
print("LoRA targets:", cfg)

LoRA targets: {'targets': ['lm_head'], 'fan_in_fan_out': False}


In [12]:
lora_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, # Es un factor de escala para el update LoRA (controla qué tan fuerte se aplica Δ𝑊.
    bias="none",
    task_type="CAUSAL_LM", #Ayuda a elegir dónde y cómo insertar LoRA correctamente. SI es modelo autoregresivo o de que tipo
    target_modules=cfg["targets"],
    fan_in_fan_out=cfg["fan_in_fan_out"] # hace que LoRA se aplique con la orientación correcta en esos casos. Es decir la orientación de las matrices
)

Congelamos el modelo student y solo tomamos parametros de el

In [13]:
student = get_peft_model(student, lora_cfg)
student.print_trainable_parameters()

trainable params: 408,200 || all params: 82,320,776 || trainable%: 0.4959


Este build_causal_dataset_streaming(...) construye un dataset para Causal Language Modeling (tipo GPT): toma texto “en streaming”, lo tokeniza, lo pega todo en una sola tira larga de tokens y luego lo corta en bloques fijos de tamaño block_size. Cada bloque se usa como:

* input_ids = tokens de entrada

* labels = los mismos tokens (HF luego hace el shift interno para predecir el siguiente token)

Convertir un corpus de texto en muchos ejemplos de longitud fija para entrenar un LM autoregresivo, y recortarlo a max_tokens para que el experimento sea corto.

Estás entrenando al modelo a predecir el siguiente token en contexto local (hasta 256/512 tokens).

El recorte existe por memoria/compute: no puedes meter secuencias infinitas, así que entrenas con ventanas.

In [14]:
def build_causal_dataset_streaming(texts, tokenizer, block_size=256, max_tokens=200_000):
    buffer, chunks, produced = [], [], 0
    for t in texts:
        t = (t or "").strip()
        if not t:
            continue
        ids = tokenizer.encode(t + "\n", add_special_tokens=False)
        buffer.extend(ids)
        while len(buffer) >= block_size:
            chunk = buffer[:block_size]
            buffer = buffer[block_size:]
            chunks.append({"input_ids": chunk, "labels": chunk.copy()})
            produced += block_size
            if produced >= max_tokens:
                return Dataset.from_list(chunks)
    return Dataset.from_list(chunks)

load_dataset("wikitext", "wikitext-2-raw-v1") carga el dataset y regresa un objeto tipo DatasetDict con varios splits:

- raw["train"]
- raw["validation"]
- raw["test"] (normalmente también está)

Cada split es un objeto Dataset y contiene una columna principal llamada "text", que es una lista de strings (líneas de texto).


In [15]:

raw = load_dataset("wikitext", "wikitext-2-raw-v1")

README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

¿Qué está pasando exactamente?

1) Tomas el texto del split correspondiente:
   - raw["train"]["text"] = lista de strings (muchas líneas de Wikipedia)
   - raw["validation"]["text"] = lista de strings para validación

2) Tokenizas con student_tokenizer:
   - Convierte cada string en una lista de ids (tokens).
   - Usas el tokenizer del student porque tus input_ids, labels y pad_id en collate_fn están basados en él,
     y en modo FREE (GPT2/DistilGPT2) el teacher comparte vocab, así que funciona bien.

3) Cortas en bloques fijos (block_size = BLOCK_SIZE):
   - Si BLOCK_SIZE=256: cada ejemplo tiene 256 tokens.
   - Si BLOCK_SIZE=512: cada ejemplo tiene 512 tokens.
   - Cada bloque produce un sample como:
     {"input_ids": [block_size ids], "labels": [block_size ids]}


In [16]:
train_ds = build_causal_dataset_streaming(raw["train"]["text"], student_tokenizer,
                                          block_size=BLOCK_SIZE, max_tokens=TRAIN_TOKENS)
val_ds   = build_causal_dataset_streaming(raw["validation"]["text"], student_tokenizer,
                                          block_size=BLOCK_SIZE, max_tokens=VAL_TOKENS)

In [17]:
print("Train samples:", len(train_ds), " Val samples:", len(val_ds))

Train samples: 782  Val samples: 79


Ese collate_fn es la función que toma una lista de ejemplos (cada uno con input_ids y labels) y los convierte en un batch de tensores del mismo tamaño usando padding, además de crear el attention_mask. Es crucial para que el Trainer pueda entrenar con batches.

In [3]:
def collate_fn(batch):
    max_len = max(len(x["input_ids"]) for x in batch)
    pad_id = student_tokenizer.pad_token_id
    input_ids, labels, attn = [], [], []
    for x in batch:
        ids = x["input_ids"]
        pad = [pad_id] * (max_len - len(ids))
        input_ids.append(ids + pad)
        labels.append(x["labels"] + [-100] * (max_len - len(ids)))
        attn.append([1]*len(ids) + [0]*len(pad))
    return {
        "input_ids": torch.tensor(input_ids),
        "labels": torch.tensor(labels),
        "attention_mask": torch.tensor(attn),
    }


Este bloque es el corazón de la distillation: calcula la pérdida total que vas a minimizar para entrenar al student, combinando:

* CE con labels reales (ground truth)

* KL para que el student imite al teacher (soft targets) usando temperatura T

In [19]:

def kd_loss(student_logits, teacher_logits, labels, T=2.0, alpha=0.2, beta=0.8):
    ce = F.cross_entropy(
        student_logits.view(-1, student_logits.size(-1)),
        labels.view(-1), ## contiene los tokens reales
        ignore_index=-100 ## evita que el padding cuente
    )

    ## construimos las soft "probabilities" del student y del teacher ("con temperatura") y luego calculamos una KL divergence para forzar
    ## que el student se parezca al profesor
    s = F.log_softmax(student_logits / T, dim=-1) ## studen logits son sin normalizar luego se hacen soft dividiendo sobre T, es decir P_{T}^{S}
    with torch.no_grad():
        t = F.softmax(teacher_logits / T, dim=-1) ## devuelve los log P_{T}^{T} esto del teacher con la misma temperatura
    kl = F.kl_div(s, t, reduction="batchmean") * (T**2) #KL(P_{T}^{T}​∥P_{T}^{S}​)
    return alpha * ce + beta * kl


La idea es: HF ya sabe hacer el loop (dataloader → forward → backward → optimizer.step → logging). Tú solo cambias cómo se calcula la loss.

outputs_s.logits tiene forma [B, L, V]

B = batch size

L = longitud (max_len del batch)

V = vocab size

In [20]:
class KDTrainer(Trainer):
    # Accept new HF kwarg: num_items_in_batch
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs["labels"]
        outputs_s = model(input_ids=inputs["input_ids"],
                          attention_mask=inputs["attention_mask"])
        student_logits = outputs_s.logits

        with torch.no_grad():
            outputs_t = teacher(
                input_ids=inputs["input_ids"].to(teacher.device),
                attention_mask=inputs["attention_mask"].to(teacher.device)
            )
            teacher_logits = outputs_t.logits.to(student_logits.device)

        loss = kd_loss(student_logits, teacher_logits, labels, T=T, alpha=ALPHA, beta=BETA)
        return (loss, {"logits": student_logits}) if return_outputs else loss

In [22]:
args = TrainingArguments(
    output_dir=OUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,# Acumulación de gradientes: hace optimizer.step() cada GRAD_ACCUM mini-batches.
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    logging_steps=25, #Cada 25 pasos imprime logs (loss, etc.).
    eval_strategy="steps",   # Le dice al Trainer que evalúe por pasos, no por epoch.
    eval_steps=200,
    save_steps=200, #Guarda un checkpoint cada 200 pasos de entrenamiento (steps, no epochs).
    save_total_limit=2, #Mantiene máximo 2 checkpoints; los viejos se borran. (Ahorra disco)
    bf16=CAN_BF16,
    fp16=(torch.cuda.is_available() and not CAN_BF16),
    gradient_checkpointing=False,  # not needed for small student; set True if you want
    report_to="none"
)

In [23]:

trainer = KDTrainer(
    model=student,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn
)


In [24]:

trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=49, training_loss=961.1742267219388, metrics={'train_runtime': 36.2012, 'train_samples_per_second': 21.601, 'train_steps_per_second': 1.354, 'total_flos': 51573824987136.0, 'train_loss': 961.1742267219388, 'epoch': 1.0})

Ese bloque hace la evaluación del modelo entrenado en tu eval_dataset y luego convierte la loss a perplexity (PPL), que es una métrica estándar en language modeling.

PPL más baja ⇒ mejor modelo (menos “sorpresa” al predecir tokens reales)

eval_loss≈α⋅CE+β⋅KL

In [25]:
eval_res = trainer.evaluate()
try:
    ppl = math.exp(eval_res["eval_loss"])
except OverflowError:
    ppl = float("inf")
print({"eval_loss": eval_res["eval_loss"], "perplexity": ppl})

{'eval_loss': 89.87802124023438, 'perplexity': 1.0802609296323229e+39}


In [26]:

SAVE_DIR = "kd-student-lora"
student.save_pretrained(SAVE_DIR)
student_tokenizer.save_pretrained(SAVE_DIR)

('kd-student-lora/tokenizer_config.json', 'kd-student-lora/tokenizer.json')

Prueba, le damos una frase inicial. El modelo va a continuarla token por token.

In [29]:
prompt = "In healthcare AI, knowledge distillation helps small models by"
inputs = student_tokenizer(prompt, return_tensors="pt").to(student.device)

In [27]:

student.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-5): 6 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Conv1D(nf=2304, nx=768)
              (c_proj): Conv1D(nf=768, nx=768)
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.1, inplace=False)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): GPT2MLP(
              (c_fc): Conv1D(nf=3072, nx=768)
              (c_proj): Conv1D(nf=768, nx=3072)
              (act): NewGELUActivation()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
        )
        (ln_f): LayerNorm((768,), eps=1e-05, e

Ese bloque es el paso de generación: le pides al student que continúe el prompt y produzca hasta 120 tokens nuevos. Aquí ya no entrenas; solo haces inferencia.

In [30]:
with torch.no_grad():
    gen = student.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=True,#Activa muestreo estocástico.
        top_p=0.9, #ordena tokens por probabilidad, toma el conjunto >= 9
        temperature=0.8, #Ojo: esta temperatura es para sampling en generación, no es la misma T de distillation.
        repetition_penalty=1.1,
        eos_token_id=student_tokenizer.eos_token_id
    )

#En generación:
#temp < 1 (0.8) → distribución más picuda → texto más “seguro”/menos random.

#temp > 1 → más creativo/random.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [31]:
print(student_tokenizer.decode(gen[0], skip_special_tokens=True))

In healthcare AI, knowledge distillation helps small models by allowing them to perform a variety of tasks.
The researchers used their data on nearly 4 million patients in the first 10 years of clinical trials (2012-2014). In 2012, 6.5 million people who performed these operations received personalized diagnosis tools that enabled physicians to determine what was needed and which treatments should be recommended for this condition. The results are even more striking than earlier studies suggesting there is an underlying reason why it can lead to disease remission or cancer survival rates. These findings suggest many new ways to help doctors get better at treating multiple sclerosis with traditional treatment options such as CT scanning systems.


### Comparando teacher vs student vs real

In [2]:
import torch
import torch.nn.functional as F
import pandas as pd

def ce_per_example(logits, labels):
    """
    logits: [B,L,V]
    labels: [B,L] con -100 en padding
    regresa CE por ejemplo (size [B]) promediado sobre tokens válidos
    """
    B, L, V = logits.shape
    log_probs = F.log_softmax(logits, dim=-1)           # [B,L,V]
    # Gather logp del token real
    labels_clamped = labels.clone()
    labels_clamped[labels_clamped == -100] = 0          # para evitar gather error
    token_logp = log_probs.gather(-1, labels_clamped.unsqueeze(-1)).squeeze(-1)  # [B,L]

    mask = (labels != -100).float()                     # [B,L]
    n_tokens = mask.sum(dim=1).clamp(min=1.0)           # [B]
    nll = -(token_logp * mask).sum(dim=1) / n_tokens    # [B]
    return nll  # CE promedio por token (por ejemplo)

# ---- 10 ejemplos del validation set ----
N = 10
batch = collate_fn([val_ds[i] for i in range(N)])
batch_s = {k: v.to(student.device) for k, v in batch.items()}
batch_t = {k: v.to(teacher.device) for k, v in batch.items()}

student.eval()
teacher.eval()
with torch.no_grad():
    s_logits = student(input_ids=batch_s["input_ids"], attention_mask=batch_s["attention_mask"]).logits
    t_logits = teacher(input_ids=batch_t["input_ids"], attention_mask=batch_t["attention_mask"]).logits.to(student.device)

ce_s = ce_per_example(s_logits, batch_s["labels"]).cpu().tolist()
ce_t = ce_per_example(t_logits, batch_s["labels"]).cpu().tolist()

# Decodifica un snippet REAL por ejemplo (primeros 200 chars)
snips = []
for i in range(N):
    ids = batch_s["input_ids"][i][batch_s["attention_mask"][i].bool()].tolist()
    txt = student_tokenizer.decode(ids, skip_special_tokens=True)
    snips.append(txt[:200].replace("\n", " ") + ("..." if len(txt) > 200 else ""))

df = pd.DataFrame({
    "idx": list(range(N)),
    "CE_student": ce_s,
    "CE_teacher": ce_t,
    "winner(lower_CE)": ["student" if ce_s[i] < ce_t[i] else "teacher" for i in range(N)],
    "real_snippet": snips
})

print("Promedios en estos 10 ejemplos:")
print({"CE_student_mean": sum(ce_s)/N, "CE_teacher_mean": sum(ce_t)/N})

df


NameError: name 'collate_fn' is not defined